In [1]:
import numpy as np
import pandas as pd

In [2]:
from env import *

In [10]:
import numpy as np
import gym
from gym import spaces

class BaseMarket(gym.Env):
    '''
    基础环境
    '''

    def __init__(self, datas,  back_length, time_limit):
        
        self.np_random = np.random.RandomState()
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=((back_length+1)*5,)
        )

        self.datas = []
        for data in datas:
            self.datas.append(data[['bid','ask','bidv','askv','volume']].copy()) #
        # self.data.loc[:,'tur'] = self.data['tur']/self.data['volume']

        self.back_length = back_length
        self.time_limit = time_limit

        self.is_eval = False
        

    def _update(self):
        '''
        更新tick数据
        '''
        # self.time = self.time+1
        # self.tick_state = self.episode_data.iloc[self.time+self.back_length-1].values
        # self.bid = self.tick_state[0]
        # self.ask = self.tick_state[1]
        # self.state = np.vstack([
        #     np.array([self.time_limit-self.time,0,0,0,0]),
        #     self.episode_data.iloc[self.time:self.time+self.back_length].values
        # ]).reshape(-1)
        pass



    def step(self, action):
        '''
        环境接收到智能体的操作信号后，首先更新下单价格与下单仓位，之后更新tick数据，判断是否成交，返回相应状态和奖励
        0:在bid0价格上等待 1:以ask价格成交
        '''
        done = False
        reward = 0
        if action == 1:
            reward = -self.ask+1
            done = True
        # elif action == 3:
        #     self.order_price = np.inf
        #     self.order_position = 1
        
        self._update()

        if action == 0 and self.bid < 0:
            reward = 1
            done = True
        elif action == 0 and self.time >= self.time_limit:
            reward = -self.ask+1
            done = True
        # elif action == 2:
        #     reward = -self.ask+1
        #     done = True
        # elif self.time >= self.time_limit:
        #     reward = -self.ask+1
        #     done = True
        # else:
        #     reward = -0.03
        #     done = False

        return self.state, reward, done, {}

    def reset(self):

        # self.time = 0
        
        # ub = int(len(self.datas)*0.99)

        # if not self.is_eval: 
        #     data = self.datas[self.np_random.randint(low=0,high=ub)]
        # else:
        #     data = self.datas[self.np_random.randint(low=ub,high=len(self.datas))]

        # begin_time = self.np_random.randint(
        #     low=self.back_length, 
        #     high=data.shape[0]-self.time_limit
        # )
        # end_time = begin_time+self.time_limit

        # self.episode_data = data.iloc[begin_time-self.back_length+1:end_time+1].copy()
        # self.target_price = self.episode_data.iloc[self.back_length-1,0]
        
        # self.episode_data.loc[:,'bid'] = self.episode_data['bid']-self.target_price
        # self.episode_data.loc[:,'ask'] = self.episode_data['ask']-self.target_price

        # self.tick_state = self.episode_data.iloc[self.back_length-1].values
        # self.bid = self.tick_state[0]
        # self.ask = self.tick_state[1]
        # self.state = np.vstack([
        #     np.array([self.time_limit,0,0,0,0]),
        #     self.episode_data.iloc[:self.back_length].values
        # ]).reshape(-1)
        self.state = 0
        return self.state

    def seed(self, seed=None):
        self.np_random = np.random.RandomState(seed)
        self.is_eval=True


In [3]:
def preprocess(tickpath, filename):
    tickdf = pd.read_csv(os.path.join(tickpath, filename),names=['date','time','ms','lastprice','volume','bid','bidv','ask','askv','opi','tur','contract'])
    tickdf['datestr'] = tickdf['date'].apply(lambda x: '%s-%s-%s'%(str(x)[:4], str(x)[4:6], str(x)[6:8]))
    tickdf['timestr'] = tickdf['time'].apply(lambda x: '%s:%s:%s'%(format(x, '06d')[:-4], format(x, '06d')[-4:-2], format(x, '06d')[-2:]))
    tickdf['timestr'] = tickdf['timestr']+'.'+tickdf['ms'].apply(lambda x: format(x, '03d'))
    tickdf = tickdf.set_index(pd.to_datetime(tickdf['datestr']+' '+tickdf['timestr']))
    tickdf.index.name = 'datetime'
    tickdf.loc[tickdf['bid']<0.9*tickdf['lastprice'], 'bid'] = np.nan
    tickdf.loc[tickdf['ask']<0.9*tickdf['lastprice'], 'ask'] = np.nan
    tickdf[['bid','ask']] = tickdf[['bid','ask']].fillna(method='ffill').fillna(tickdf.iloc[0]['lastprice'])
    tickdf['midprice'] = (tickdf['ask']+tickdf['bid'])/2.0

    tdata = tickdf[['bid','ask','bidv','askv','volume']]
    std = tdata.iloc[0,0]
    tdata.loc[:,'bid'] = tdata['bid']-std
    tdata.loc[:,'ask'] = tdata['ask']-std

    def norm(x):
        return (x-x.mean())/x.std()

    tdata.loc[:,'askv'] = norm(tdata.askv)
    tdata.loc[:,'bidv'] = norm(tdata.bidv)
    tdata.loc[:,'volume'] = norm(tdata.volume)
    return tdata

In [4]:
import os
cwd = '/Data/database/data_zltick/rb'
datas = []
for _,__, files in os.walk(cwd):
    for file in files:
        if int(file[:4])<2020:
            continue
        data = preprocess(cwd, file)
        datas.append(data)

In [5]:
# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots()

# ax2 = ax1.twinx()
# ax1.plot(tdata.bid[500:800])
# ax2.plot(tdata.volume[500:800], color='red')

# plt.show()

In [6]:
env = BaseMarket(datas=datas,back_length=100,time_limit=600)

In [11]:
env = BaseMarket(datas=datas,back_length=100,time_limit=600)
tot = 0
env.seed(567)
for i in range(1024):
    done = False
    state = env.reset()
    action = 0
    while not done:
        state, reward, done, _ = env.step(action)
        if state[0] == 1:
            action = 1
    tot += reward
print(tot/1024)

AttributeError: 'BaseMarket' object has no attribute 'time'

In [8]:
env.seed(567)
tot = 0
for i in range(1024):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1024)

-0.0478515625


In [9]:
import elegantrl.agent as agent
from elegantrl.config import Arguments
env_args = {
    'env_num': 1,
    'env_name': 'executioner',
    'max_step': 600,
    'state_dim': (200+1)*5,
    'action_dim': 2,
    'if_discrete': True,
    'target_return': 0.5
}

In [10]:
env = BaseMarket(datas=datas,back_length=200,time_limit=600)
args = Arguments(agent.AgentD3QN,env=env, env_args=env_args)

In [11]:
from elegantrl.run import train_and_evaluate

In [12]:
args.target_step = args.max_step * 50
args.eval_times = 2 ** 10
args.net_dim = 1024
args.batch_size = 1024
args.gamma = 1
args.explore_rate = 0.1
args.seed = 567
args.learning_rate = 2**-15
train_and_evaluate(args)

| Arguments Remove cwd: ./executioner_D3QN_0
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  3.00e+04    0.08 |
0  3.00e+04    0.08 |    0.08    3.3    110   188 |    0.00   0.03   0.21
0  9.01e+04    0.08 |    0.08    3.3    110   188 |    0.00   0.05   0.17
0  2.40e+05    0.12 |
0  2.40e+05    0.12 |    0.12    2.9     93   167 |    0.00   0.04   0.25
0  3.30e+05    0.12 |    0.03    0.8      5     6 |    0.01   0.06  -0.50
0  5.40e+05    0.15 |
0  5.40e+05    0.15 |    0.15    0.8      7    21 |    0.00   0.10  -0.42
0  7.50e+05    0.21 |
0  7.50e+05    0.21 |    0.21    1.9     49   104 |    0.01   0.06  -0.15
0  8.70e+05    0.25 |
0  8.70e+05    0.25 |    0.25    2.2     57   124 |    0.02   0.05   0.08
0  9.90e+05    0.25 |    0.20    1.9     45    78 |    0.01   0.05  -0.05


In [34]:
# age = args.agent().save_or_load_agent(cwd='./executioner_D3QN_0', if_save=False)
from elegantrl.run import *

In [35]:
import torch
torch.set_grad_enabled(False)
# args.init_before_training()
gpu_id = args.learner_gpus

'''init'''
env = build_env(args.env, args.env_func, args.env_args)

age = init_agent(args, gpu_id, env)

In [36]:
age.save_or_load_agent('./executioner_D3QN_other', if_save=False)

In [37]:
# for seed in range(100):
seed = 123
env.seed(seed)
tot = 0
tmp = 0
for i in range(100000):
    done = False
    state = env.reset()
    steps = 0
    # print(state.reshape((-1, 5)))
    while not done:
        steps += 1
        ten_s = torch.as_tensor(state, dtype=torch.float32).unsqueeze(0)
        ten_a = age.act(ten_s)
        # print(ten_a)
        ten_a = ten_a.argmax(dim=1)
        state, reward, done, _ = env.step(ten_a[0].numpy())  # different
        # print(state[-5:])
        # print(reward)
    tot += reward
    tmp += reward
    if i % 1000 == 0:
        print(f'1000 times avg: {tmp/1000}')
        tmp = 0
    # env.episode_data[:300].to_csv('tmp.csv')
    # break
    # print(steps)
print(tot/100000)

1000 times avg: 0.001
1000 times avg: 0.319
1000 times avg: 0.262
1000 times avg: 0.291
1000 times avg: 0.287
1000 times avg: 0.248
1000 times avg: 0.372
1000 times avg: 0.314
1000 times avg: 0.292
1000 times avg: 0.332
1000 times avg: 0.194
1000 times avg: 0.308
1000 times avg: 0.285
1000 times avg: 0.18
1000 times avg: 0.309
1000 times avg: 0.336
1000 times avg: 0.291
1000 times avg: 0.272
1000 times avg: 0.295
1000 times avg: 0.318
1000 times avg: 0.334
1000 times avg: 0.301
1000 times avg: 0.298
1000 times avg: 0.33
1000 times avg: 0.253
1000 times avg: 0.316
1000 times avg: 0.311
1000 times avg: 0.3
1000 times avg: 0.281
1000 times avg: 0.335
1000 times avg: 0.25
1000 times avg: 0.343
1000 times avg: 0.332
1000 times avg: 0.333
1000 times avg: 0.31
1000 times avg: 0.329
1000 times avg: 0.293
1000 times avg: 0.359
1000 times avg: 0.321
1000 times avg: 0.257
1000 times avg: 0.368
1000 times avg: 0.335
1000 times avg: 0.261
1000 times avg: 0.169
1000 times avg: 0.239
1000 times avg: 

In [21]:
for seed in range(100):
    tot = 0
    env.seed(seed)
    for i in range(1000):
        done = False
        state = env.reset()
        action = 0
        while not done:
            state, reward, done, _ = env.step(action)
            if state[0] == 1:
                action = 1
        tot += reward
    print(tot/1000)

0.193
-0.09
-0.106


KeyboardInterrupt: 

In [38]:
tot = 0
env.seed(1)
for i in range(1000):
    done = False
    state = env.reset()
    action = 1
    while not done:
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1000)

-0.046


In [41]:
tot = 0
env.seed(1)
for i in range(10000):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/10000)

-0.0302


In [28]:
tmp = np.load(r'executioner_D3QN_0\replay_0.npz')

In [40]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('tkagg')
plt.plot(data['bid'])
plt.show()

In [35]:
tmp['buf_other']

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.99,  2.  ],
       [ 0.  ,  0.99,  2.  ],
       [-0.  ,  0.  ,  2.  ]], dtype=float16)

In [5]:
a = [(1,2,3),(3,4,5),(5,6,5),(7,8,5)]
b = list(map(list, zip(*a)))

In [7]:
import torch
b

[[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 5, 5]]